In [64]:
import pandas as pd
import numpy as np
import yaml
debug=False
from __future__ import print_function
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [2]:
sampleNumber="01-X"
samplePrefix="MGC-150815-extend-%s-paradigmes"%sampleNumber
analysisPrefix="MGC-150815-extend-%s"%sampleNumber

In [5]:
SILVER=pd.read_csv("2015-Data/"+analysisPrefix+'-Silver.csv',sep=";",encoding="utf8")
del SILVER["Unnamed: 0"]

In [59]:
GOLD=pd.read_csv("2015-Data/"+analysisPrefix+'-Gold.csv',sep=";",encoding="utf8")
del GOLD["Unnamed: 0"]

In [47]:
paradigmeSILVER=SILVER.groupby("lexeme").agg(lambda x: list(set(x.dropna().values))).reset_index()

In [57]:
sampleCases=paradigmeSILVER.columns.values.tolist()
for element in [u"lexeme",u"ajouts",u"score"]:
    sampleCases.remove(element)
analyseCases=sampleCases

In [108]:
def compareGOLD(row):
    global n
#    if row["lexeme"]=="dodo": 
#        debug=True
#    else:
#        debug=False
    if n%250==0: print (n,end=", ")
    n+=1
    ligne={k:",".join(row[k]) for k in row.keys() if not k in ["lexeme","score","ajouts"]}
    if n<5: print (ligne)
    lexeme=row["lexeme"]
    score=float(row["score"][0])
    ajouts=int(row["ajouts"][0])
    if not lexeme in conjugaisons:
        conjugaisons[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    if not lexeme in cliquesTypes:
        cliquesTypes[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    identiques=[]
    differents=[]
    missing=[]
    over=[]
    for case in analyseCases:
        if debug:    
            print (lexeme,case)
            print (GOLD[GOLD["lexeme"]==lexeme][case].item())
            print ((GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]))
#        if not case in row:
        if not case in ligne:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("absence",case)
#        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
        elif ligne[case]=="" and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
#        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
        elif ligne[case]!="" and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
#            over.append(u"%s : %s ≠ Ø" % (case,row[case]))
            over.append(u"%s : %s ≠ Ø" % (case,ligne[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
#        elif (row[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
        elif (ligne[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
            if debug: print ("egal")
#            identiques.append(u"%s : %s" % (case,row[case]))
            identiques.append(u"%s : %s" % (case,ligne[case]))
            if debug: print ("valeurs id")
#        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
        elif ligne[case]=="" and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
            if debug: print ("NaNs")
            identiques.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
#            differents.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            differents.append(u"%s : %s ≠ %s" % (case, ligne[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if differents:
        conjugaisons[lexeme]["diff"].append(differents)
        cliquesTypes[lexeme]["diff"].append((ligne,score,ajouts))
        if debug: print ("DIFF",len(differents),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: print (", ".join(differents))
    if missing and over:
        conjugaisons[lexeme]["overmiss"].append(over+missing)
        cliquesTypes[lexeme]["overmiss"].append((ligne,score,ajouts))
        if debug: print ("OVERMISS", len(over), len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
            print (", ".join(over))
    elif missing:
        conjugaisons[lexeme]["miss"].append(missing)
        cliquesTypes[lexeme]["miss"].append((ligne,score,ajouts))
        if debug: print ("MISS", len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
    elif over:
        conjugaisons[lexeme]["over"].append(over)
        cliquesTypes[lexeme]["over"].append((ligne,score,ajouts))
        if debug: print ("OVER", len(over),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(over))
    if identiques:
        conjugaisons[lexeme]["ok"].append(identiques)
        cliquesTypes[lexeme]["ok"].append((ligne,score,ajouts))
        if debug: print ("OK", len(identiques), len(identiques)-nombreElements[lexeme],end=saut)
        if debug1: print (", ".join(identiques))
#    print ()
    return lexical

In [109]:
%%time
debug=False
debug1=False
saut="\n"
n=0
conjugaisons={}
cliquesTypes={}
print (", ".join(sampleCases))
#paradigmesCLIQUES.apply(compareGOLD,axis=1)
paradigmeSILVER.apply(compareGOLD,axis=1)

ai1P, ai1S, ai2P, ai2S, ai3P, ai3S, fi1P, fi1S, fi2P, fi2S, fi3P, fi3S, ii1P, ii1S, ii2P, ii2S, ii3P, ii3S, inf, is1P, is1S, is2P, is2S, is3P, is3S, pI1P, pI2P, pI2S, pP, pc1P, pc1S, pc2P, pc2S, pc3P, pc3S, pi1P, pi1S, pi2P, pi2S, pi3P, pi3S, ppFP, ppFS, ppMP, ppMS, ps1P, ps1S, ps2P, ps2S, ps3P, ps3S
0, {u'ii1P': u'abEsj\xf4', u'ii1S': u'abEsE', u'pc1P': '', u'pP': u'abEs\xe2', u'ai3S': u'abEsa', u'ai3P': u'abEsEr', u'is3P': '', u'is3S': '', u'ps3S': u'abEs', u'ii3S': u'abEsE', u'ai1P': '', u'inf': u'abEse', u'ii3P': u'abEsE', u'pi2S': u'abEs', u'ai1S': u'abEsE', u'pi2P': u'abEse', u'ppFS': u'abEse', u'ppFP': u'abEse', u'is2S': '', u'is2P': '', u'ps3P': u'abEs', u'ps1S': u'abEs', u'pI2S': u'abEs', u'fi2P': '', u'fi2S': u'abEs6ra', u'pI2P': '', u'ps2P': '', u'ps2S': u'abEs', u'ps1P': '', u'ai2P': '', u'ai2S': '', u'pc2P': '', u'pc2S': u'abEs6rE', u'ii2P': '', u'pi1S': u'abEs', u'pi1P': u'abEs\xf4', u'ii2S': '', u'pi3P': u'abEs', u'pi3S': u'abEs', u'ppMS': u'abEse', u'pc3S': u'abEs6rE', 

In [110]:
yaml.safe_dump(conjugaisons, file("2015-Data/"+samplePrefix+dateheure()+".yaml",'w'), encoding='utf-8', allow_unicode=True)
yaml.safe_dump(cliquesTypes, file("2015-Data/"+samplePrefix+"cliques-"+dateheure()+".yaml",'w'), encoding='utf-8', allow_unicode=True)

In [111]:
precisions={"bon":0,"mauvais":0}
for verbe in cliquesTypes:
    if debug: print (verbe)
    for cle in cliquesTypes[verbe]:
        if debug: print (cle)
        for element in cliquesTypes[verbe][cle]:
            if debug: print (element[1],element[2])
            if cle in ["ok"]:
                precisions["bon"]+=element[2]
            elif cle in ["diff"]:
                precisions["mauvais"]+=element[2]
precisions

{'bon': 17831, 'mauvais': 1472}

In [112]:
float(precisions["bon"])/(precisions["bon"]+precisions["mauvais"])

0.9237424234574937

In [113]:
basesMesures={"total":0,"bon":0,"mauvais":0}
for verbe in conjugaisons:
    for cat in conjugaisons[verbe]:
        if conjugaisons[verbe][cat]:
#            print (verbe,cat,len(conjugaisons[verbe][cat][0]))
            if cat=="miss":
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
            elif cat=="ok":
                basesMesures["bon"]+=len(conjugaisons[verbe][cat][0])
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
            elif cat=="diff":
                basesMesures["mauvais"]+=len(conjugaisons[verbe][cat][0])
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
            elif cat=="overmiss":
                basesMesures["mauvais"]+=len(conjugaisons[verbe][cat][0])
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
basesMesures

{'bon': 22120, 'mauvais': 2341, 'total': 44508}

In [114]:
precisionManip=float(basesMesures["bon"])/(basesMesures["bon"]+basesMesures["mauvais"])
precisionManip

0.904296635460529

In [115]:
rappelManip=float(basesMesures["bon"])/(basesMesures["total"])
rappelManip

0.4969893052934304